In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
PATH = ""

df = pd.read_csv(os.path.join(PATH, "QAResultData.csv"))
data = np.array(df.values)
df_keys = np.array([[eval(i)[0], eval(i)[1]] for i in data[:,0]])

 
logi_pool = [(file_name[0][:2] == 'M'+str(1))&(file_name[0][2] == 'M')&('SRM' not in file_name[0]) for file_name in df_keys] 

parameter = df_keys[logi_pool][:,1]
Ct_err = data[logi_pool][:,2]
Cc_err = data[logi_pool][:,3]
CD = data[logi_pool][:,4]   
SP = data[logi_pool][:,5]

train_X = np.array([[CD[i], SP[i]] for i in range(0, len(CD), 2)])
#(Ct_err1*wext2 - Ct_err2*wext1)/(Ct_err1 - Ct_err2)
train_Y = np.array([(Ct_err[i]*parameter[i+1][0]-Ct_err[i+1]*parameter[i][0])/(Ct_err[i]-Ct_err[i+1]) for i in range(0, len(Ct_err), 2)])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [84]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

regressor = DecisionTreeRegressor(random_state=0, min_samples_leaf = 60)

In [85]:
regressor.fit(train_X, train_Y)

DecisionTreeRegressor(min_samples_leaf=60, random_state=0)

In [86]:
regressor.tree_.feature

array([ 0,  0,  1, -2, -2, -2,  0, -2,  1,  1, -2, -2, -2], dtype=int64)

In [87]:
regressor.tree_.threshold

array([425. , 359. , 334.5,  -2. ,  -2. ,  -2. , 489. ,  -2. , 438.5,
       314.5,  -2. ,  -2. ,  -2. ])

In [88]:
regressor.tree_.children_left

array([ 1,  2,  3, -1, -1, -1,  7, -1,  9, 10, -1, -1, -1], dtype=int64)

In [89]:
regressor.tree_.children_right

array([ 6,  5,  4, -1, -1, -1,  8, -1, 12, 11, -1, -1, -1], dtype=int64)

In [90]:
len(regressor.tree_.children_right)

13

In [91]:
def split(array, dim, thred):
    if dim == 0:
        return [[np.min(array[0]), thred], array[1]],  [[thred, np.max(array[0])], array[1]]
    elif dim == 1:
        return [array[0], [np.min(array[1]), thred]],  [array[0], [thred, np.max(array[1])]]

In [92]:
interval = [[0,1000], [0,1000]]
left_children = [interval]
right_children = [interval]
right_remove_idx = False
left_remove_idx = False
parent = []
region = []
LEFT_FLAG = False

for index_of_node in range(0,len(regressor.tree_.threshold)):
    if regressor.tree_.threshold[index_of_node] != -2:
        
#         print(right_children)
        
        if LEFT_FLAG == False:
            
            FROM_LEFT_FLAG = False
            
            right, left = split(right_children[-1], regressor.tree_.feature[index_of_node], regressor.tree_.threshold[index_of_node])
            
            parent.append([0,1,len(right_children)-1, False, False])
            right_children.append(right)
            left_children.append(left)
#             print(left_children)
            FROM_RIGHT_FLAG = True
        else:
            FROM_RIGHT_FLAG = False
            
            right, left = split(left_children[-1], regressor.tree_.feature[index_of_node], regressor.tree_.threshold[index_of_node])
            
            
            
            parent.append([1, 0,len(left_children)-1, False, False])
            
            right_children.append(right)
            left_children.append(left)
            LEFT_FLAG = False
            FROM_LEFT_FLAG = True
        print(index_of_node, parent)
    else:
        print(index_of_node, parent)
        
        if regressor.tree_.threshold[index_of_node-1] != -2:
            
            region.append(right_children[-1])
            
            parent_idx = parent[-1][0:3]
#             parent_remove = parent[-1][3:]
            
            right_children.pop()
            LEFT_FLAG = True
#             parent_remove[1] = True
            parent[-1][4] = True
        else:
            
            region.append(left_children[-1])
            
            parent_idx = parent[-1][0:3]
#             parent_remove = parent[-1][3:]

            left_children.pop()
            parent[-1][3] = True
     
    
#         print(left_remove_idx, right_remove_idx)
        while (parent[-1][3:][0] == True) & (parent[-1][3:][1] == True):
#             if len(parent == len(regressor.tree_.threshold)-1:
#                 break:
            
            if parent_idx[0] == 1: #left
                left_children.pop(parent_idx[2])
                parent.pop()
                if (index_of_node == len(regressor.tree_.feature)-1) & (len(parent) == 0):
                    break
                else:
                    parent_idx = parent[-1][0:3]
#                 parent_remove = parent[-1][3:]
                
                    parent[-1][3] = True
            else:
                right_children.pop(parent_idx[2])
                parent.pop()
                if (index_of_node == len(regressor.tree_.feature)-1) & (len(parent) == 0):
                    break
                else:
                    parent_idx = parent[-1][0:3]
    #                 parent_remove = parent[-1][3:]

                    parent[-1][4] = True

        else:
            continue


0 [[0, 1, 0, False, False]]
1 [[0, 1, 0, False, False], [0, 1, 1, False, False]]
2 [[0, 1, 0, False, False], [0, 1, 1, False, False], [0, 1, 2, False, False]]
3 [[0, 1, 0, False, False], [0, 1, 1, False, False], [0, 1, 2, False, False]]
4 [[0, 1, 0, False, False], [0, 1, 1, False, False], [0, 1, 2, False, True]]
5 [[0, 1, 0, False, False], [0, 1, 1, False, True]]
6 [[0, 1, 0, False, True], [1, 0, 1, False, False]]
7 [[0, 1, 0, False, True], [1, 0, 1, False, False]]
8 [[0, 1, 0, False, True], [1, 0, 1, False, True], [1, 0, 2, False, False]]
9 [[0, 1, 0, False, True], [1, 0, 1, False, True], [1, 0, 2, False, False], [0, 1, 1, False, False]]
10 [[0, 1, 0, False, True], [1, 0, 1, False, True], [1, 0, 2, False, False], [0, 1, 1, False, False]]
11 [[0, 1, 0, False, True], [1, 0, 1, False, True], [1, 0, 2, False, False], [0, 1, 1, False, True]]
12 [[0, 1, 0, False, True], [1, 0, 1, False, True], [1, 0, 2, False, True]]


In [93]:
right_children

[]

In [94]:
left_children

[[[0, 1000], [0, 1000]]]

In [96]:
print(tree.export_text(regressor))

|--- feature_0 <= 425.00
|   |--- feature_0 <= 359.00
|   |   |--- feature_1 <= 334.50
|   |   |   |--- value: [0.94]
|   |   |--- feature_1 >  334.50
|   |   |   |--- value: [-3.27]
|   |--- feature_0 >  359.00
|   |   |--- value: [-4.90]
|--- feature_0 >  425.00
|   |--- feature_0 <= 489.00
|   |   |--- value: [-8.91]
|   |--- feature_0 >  489.00
|   |   |--- feature_1 <= 438.50
|   |   |   |--- feature_1 <= 314.50
|   |   |   |   |--- value: [-10.12]
|   |   |   |--- feature_1 >  314.50
|   |   |   |   |--- value: [-12.35]
|   |   |--- feature_1 >  438.50
|   |   |   |--- value: [-14.47]



In [95]:
region

[[[0.0, 359.0], [0, 334.5]],
 [[0.0, 359.0], [334.5, 1000]],
 [[359.0, 425.0], [0, 1000]],
 [[425.0, 489.0], [0, 1000]],
 [[489.0, 1000.0], [0.0, 314.5]],
 [[489.0, 1000.0], [314.5, 438.5]],
 [[489.0, 1000.0], [438.5, 1000]]]